In [ ]:
"""
05_optimization_single_charge.ipynb

Problem: Minimize total EV charging cost over 72 hours.
Inputs: Hourly price forecast, battery and charger constraints.
Business goal: Find optimal charging hours to reach full battery at lowest cost.

Author: [Your Name]
Date: 2025-11-02
"""
# %% Setup and Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from datetime import datetime, timedelta

forecast_horizon = 72

X_test = test[feature_cols]
ts_test = test['timestamp']              # make sure this is a datetime column

# 72-hour price forecast from the best model (LightGBM)
price_forecast = lgb_model.predict(X_test.iloc[:forecast_horizon])
price_forecast = price_forecast.astype(float)  # ensure numeric type

# matching time index for those 72 rows
time_index = ts_test.iloc[:forecast_horizon].to_list()

# EV parameters
EV_BATTERY_KWH = 60      # Total battery (kWh)
EV_INITIAL_SOC = 0.10    # Initial state of charge (10%)
EV_TARGET_SOC = 1.00     # Target SOC (full)
EV_CHARGE_POWER_KW = 11  # Max charge rate (kW/hour)
EFFICIENCY = 0.98        # Charging efficiency

# How much to charge:
kWh_to_charge = EV_BATTERY_KWH * (EV_TARGET_SOC - EV_INITIAL_SOC)

print(f"Energy to charge: {kWh_to_charge:.1f} kWh (from {EV_INITIAL_SOC:.0%} to {EV_TARGET_SOC:.0%})")

# %% Optimization Model: PuLP

model = pulp.LpProblem('EV_Charging_Schedule', pulp.LpMinimize)
# Continuous per-hour charging variable
charge_kw = pulp.LpVariable.dicts('charge_kw', range(72), lowBound=0, upBound=EV_CHARGE_POWER_KW, cat='Continuous')

# Objective: minimize cost
model += pulp.lpSum((price_forecast[i]/1000) * charge_kw[i] for i in range(72))  # EUR

# Constraint: total kWh charged (accounting for efficiency)
model += pulp.lpSum(charge_kw[i] * EFFICIENCY for i in range(72)) >= kWh_to_charge

# Per hour charge limits (handled by variable bounds)
for i in range(72):
    model += charge_kw[i] <= EV_CHARGE_POWER_KW
    model += charge_kw[i] >= 0

# Solve
model.solve()

# %% Extract Schedule and Cost

charging_schedule = np.array([charge_kw[i].varValue for i in range(72)])
total_charged = charging_schedule.sum() * EFFICIENCY
total_cost = np.sum(charging_schedule * price_forecast / 1000)

print(f"Optimal total energy charged: {total_charged:.2f} kWh")
print(f"Total cost: €{total_cost:.2f}")

# %% Comparison: Flat (Uniform) Charging

flat_per_hour = kWh_to_charge / 72
flat_schedule = np.ones(72) * flat_per_hour
flat_cost = np.sum(flat_schedule * price_forecast / 1000)

print(f"Flat charging cost: €{flat_cost:.2f}")
print(f"Cost savings (optimized): €{flat_cost - total_cost:.2f}")

# %% Visualization

plt.figure(figsize=(18,7))
plt.subplot(2,1,1)
plt.plot(time_index, price_forecast, '-', label='Price Forecast (EUR/MWh)')
plt.bar(time_index, charging_schedule, width=1/1.5, alpha=0.7, label='Optimal Charging (kW)')
plt.plot(time_index, flat_schedule, '--', color='black', label='Flat Charging (kW)')
plt.ylabel('Price/Energy')
plt.legend()
plt.title('Optimized vs. Flat EV Charging (72h)')

plt.subplot(2,1,2)
plt.plot(time_index, np.cumsum(charging_schedule)*EFFICIENCY, label='Cumulative Charged (kWh)')
plt.axhline(kWh_to_charge, color='red', linestyle='--', label='Required kWh')
plt.ylabel('kWh')
plt.xlabel('Datetime')
plt.legend()

plt.tight_layout()
plt.savefig('data/processed/05_single_charge_optimization.png')
plt.show()

# %% Export schedule

schedule_df = pd.DataFrame({
    'timestamp': time_index,
    'price_eur_mwh': price_forecast,
    'charge_kW_optimal': charging_schedule,
    'charge_kW_flat': flat_schedule
})
schedule_df.to_csv('data/processed/ev_schedule_single_charge.csv', index=False)
print("✓ Charging schedule exported to data/processed/ev_schedule_single_charge.csv")

# %% Summary

print(f"\n*** SINGLE-CHARGE OPTIMIZATION COMPLETE ***")
print(f"  Optimal schedule total cost: €{total_cost:.2f}")
print(f"  Flat charging total cost:    €{flat_cost:.2f}")
print(f"  Cost savings:                €{flat_cost - total_cost:.2f}\n")
print("Ready to show results to recruiter or extend to Problem 2.")



In [ ]:
import pandas as pd
from datetime import datetime

forecast_start = datetime(2025, 11, 2, 18)  # replace with your last-observation + 1h if known
time_index = pd.date_range(start=forecast_start, periods=72, freq='H')

In [ ]:
def build_lgbm_forecast(lgb_model, test_df, feature_cols, horizon=72, ts_col='timestamp'):
    Xh = test_df[feature_cols].iloc[:horizon]
    yh = lgb_model.predict(Xh).astype(float)
    if ts_col in test_df.columns:
        th = pd.to_datetime(test_df[ts_col].iloc[:horizon]).to_list()
    else:
        th = None
    return yh, th

price_forecast, time_index = build_lgbm_forecast(lgb_model, test, feature_cols, 72, 'timestamp')